In [ ]:
import requests
import pandas as pd
import datetime
import os
import schedule
import time

# Configuration
base_url = 'http://sc-e.fr/docs/'
repertoire_sauvegarde = r'E:\Users\Desktop\projet_3\projet_3'

# Fonction pour télécharger un fichier
def telecharger_fichier(url_fichier, chemin_sauvegarde):
    response = requests.get(url_fichier)
    if response.status_code == 200:
        with open(chemin_sauvegarde, 'wb') as file:
            file.write(response.content)
        print(f'Téléchargé avec succès: {chemin_sauvegarde}')
    else:
        print(f'Échec du téléchargement: {url_fichier}, Code de statut: {response.status_code}')

# Fonction pour charger un fichier CSV en DataFrame
def charger_csv(chemin_fichier):
    if os.path.exists(chemin_fichier):
        return pd.read_csv(chemin_fichier)
    else:
        print(f'Fichier non trouvé: {chemin_fichier}')
        return pd.DataFrame()

# Fonction principale
def principal():
    # Vérifier si le répertoire de sauvegarde existe, sinon, le créer
    if not os.path.exists(repertoire_sauvegarde):
        os.makedirs(repertoire_sauvegarde)
    
    # Obtenir la date du jour
    aujourdhui = datetime.date.today()
    date_str = aujourdhui.strftime('%Y-%m-%d')
    hier = aujourdhui - datetime.timedelta(days=1)
    hier_str = hier.strftime('%Y-%m-%d')

    # Construire les URLs des fichiers à télécharger
    url_logs = f'{base_url}logs_vols_{date_str}.csv'
    url_degradations = f'{base_url}degradations_{date_str}.csv'

    # Construire les chemins de sauvegarde
    chemin_logs = os.path.join(repertoire_sauvegarde, f'logs_vols_{date_str}.csv')
    chemin_degradations = os.path.join(repertoire_sauvegarde, f'degradations_{date_str}.csv')
    
    # Télécharger les fichiers
    telecharger_fichier(url_logs, chemin_logs)
    telecharger_fichier(url_degradations, chemin_degradations)

    # Charger les nouveaux fichiers avec pandas seulement s'ils existent
    nouveaux_logs = charger_csv(chemin_logs)
    nouvelles_degradations = charger_csv(chemin_degradations)

    if nouveaux_logs.empty or nouvelles_degradations.empty:
        print('Nouveaux fichiers non disponibles ou vides.')
        return

    # Construire les chemins des anciens fichiers
    ancien_chemin_logs = os.path.join(repertoire_sauvegarde, f'logs_vols_{hier_str}.csv')
    ancien_chemin_degradations = os.path.join(repertoire_sauvegarde, f'degradations_{hier_str}.csv')

    # Charger les anciens fichiers
    anciens_logs = charger_csv(ancien_chemin_logs)
    anciennes_degradations = charger_csv(ancien_chemin_degradations)

    # Combiner les nouvelles données avec les anciennes
    logs_combines = pd.concat([anciens_logs, nouveaux_logs]).drop_duplicates().reset_index(drop=True)
    degradations_combines = pd.concat([anciennes_degradations, nouvelles_degradations]).drop_duplicates().reset_index(drop=True)

    # Sauvegarder les fichiers combinés
    chemin_logs_combines = os.path.join(repertoire_sauvegarde, f'logs_vols_{date_str}.csv')
    chemin_degradations_combines = os.path.join(repertoire_sauvegarde, f'degradations_{date_str}.csv')

    logs_combines.to_csv(chemin_logs_combines, index=False)
    degradations_combines.to_csv(chemin_degradations_combines, index=False)

    print(f'Fichiers mis à jour et sauvegardés dans: {repertoire_sauvegarde}')

# Planifie l'exécution de la fonction toutes les 10 minutes
schedule.every(10).minutes.do(principal)

while True:
    schedule.run_pending()
    time.sleep(1)

# Préparation des Données

In [3]:
import pandas as pd

# Charger le fichier Aeronefs
df_logs_vols = pd.read_csv( r'E:\Users\Desktop\projet_3\projet_3\logs_vols_2024-06-05.csv')
# Charger le fichier composants
df_degradations = pd.read_csv( r'E:\Users\Desktop\projet_3\projet_3\degradations_2024-06-05.csv')
# Charger le fichier degradations
df_Aeronefs = pd.read_csv( r'E:\Users\Desktop\projet_3\projet_3_0\projet_3\Aeronefs_2023-11-07.csv')
# Charger le fichier logs_vols
df_composants = pd.read_csv( r'E:\Users\Desktop\projet_3\projet_3_0\projet_3\Composants_2023-11-07.csv')



In [28]:
df_Aeronefs 


,ref_aero,type_model,debut_service,last_maint,en_maintenance,end_maint
0,E175_4124,E175,2003-10-01,2023-11-01 00:00:00,False,NaN
1,E175_6334,E175,2010-02-18,2023-11-01 00:00:00,True,2023-11-08 14:13:00
2,B777_1214,B777,2003-10-21,2023-11-06 00:00:00,True,2023-11-13 14:13:00
3,CRJ900_0813,CRJ900,2002-08-01,2023-11-03 00:00:00,True,2023-11-10 14:13:00
4,A340_0268,A340,2003-01-12,2023-11-04 00:00:00,False,NaN
...,...,...,...,...,...,...
227,B737_4516,B737,2001-10-07,2023-10-08 00:00:00,False,NaN
228,B747_2319,B747,2002-09-27,2023-10-25 00:00:00,False,NaN
229,A330_6228,A330,2012-05-23,2023-10-22 00:00:00,False,NaN
230,A350_1769,A350,2022-03-20,2023-11-01 00:00:00,True,2023-11-08 14:13:00


In [29]:
df_composants 


,ref_compo,categorie,aero,desc,lifespan,taux_usure_actuel,cout
0,REAE175-E175_4124-0,Composants Critiques,E175_4124,Réacteur gauche,11950,69.234150,19845
1,REAE175-E175_4124-1,Composants Critiques,E175_4124,Réacteur droit,14014,65.181958,17416
2,SYSE175-E175_4124-2,Composants Critiques,E175_4124,Système de navigation,12519,17.981443,16184
3,ORDE175-E175_4124-3,Composants Critiques,E175_4124,Ordinateur de vol,11612,26.147164,18574
4,AUTE175-E175_4124-4,Composants Critiques,E175_4124,Autopilote,10655,11.473035,18769
...,...,...,...,...,...,...,...
10435,SYSB767-B767_6363-40,Composants Secondaires,B767_6363,Systèmes d'oxygène d'urgence,12602,26.476842,4344
10436,PORB767-B767_6363-41,Composants Secondaires,B767_6363,Portes passagers,11227,19.875467,2970
10437,HUBB767-B767_6363-42,Composants Secondaires,B767_6363,Hublots,14930,17.766530,2567
10438,ECLB767-B767_6363-43,Composants Secondaires,B767_6363,Éclairage de la cabine,11837,28.869277,4034


In [30]:
df_logs_vols 


,ref_vol,aero_linked,jour_vol,time_en_air,sensor_data,etat_voyant
0,V06683852,B737_0940,2024-06-05,0.6,"{'temp': '17.8°C', 'pressure': '904.2 hPa', 'v...",0
1,V03551025,E175_6879,2024-06-05,5.2,"{'temp': '-1.3°C', 'pressure': '1029.6 hPa', '...",0
2,V04512442,A320_2562,2024-06-05,7.9,"{'temp': '16.4°C', 'pressure': '1071.7 hPa', '...",0
3,V06831692,A320_1884,2024-06-05,7.4,"{'temp': '-24.3°C', 'pressure': '905.9 hPa', '...",1
4,V05264659,A321_3033,2024-06-05,3.6,"{'temp': '5.3°C', 'pressure': '1053.8 hPa', 'v...",0
...,...,...,...,...,...,...
134,V01078793,CRJ700_2863,2024-06-05,7.4,"{'temp': '-7.2°C', 'pressure': '927.0 hPa', 'v...",0
135,V00538128,A321_0181,2024-06-05,2.8,"{'temp': '2.8°C', 'pressure': '867.2 hPa', 'vi...",0
136,V00228114,E170_2583,2024-06-05,5.5,"{'temp': '-10.2°C', 'pressure': '998.9 hPa', '...",0
137,V09501055,A350_4039,2024-06-05,5.6,"{'temp': '5.0°C', 'pressure': '932.7 hPa', 'vi...",0


In [31]:
df_degradations

,ref_deg,linked_aero,compo_concerned,usure_nouvelle,measure_day,need_replacement
0,D003661,E170_6353,REAE170-E170_6353-0,54.208480,2024-06-04,False
1,D009402,E170_6353,REAE170-E170_6353-1,51.052144,2024-06-04,False
2,D000812,E170_6353,SYSE170-E170_6353-2,17.457904,2024-06-04,False
3,D006665,E170_6353,ORDE170-E170_6353-3,7.373504,2024-06-04,False
4,D001163,E170_6353,AUTE170-E170_6353-4,43.057968,2024-06-04,False
...,...,...,...,...,...,...
10525,D006322,CRJ700_2988,SYSCRJ700-CRJ700_2988-40,53.688232,2024-06-05,False
10526,D005497,CRJ700_2988,PORCRJ700-CRJ700_2988-41,16.759548,2024-06-05,False
10527,D005625,CRJ700_2988,HUBCRJ700-CRJ700_2988-42,36.482392,2024-06-05,False
10528,D003657,CRJ700_2988,ECLCRJ700-CRJ700_2988-43,37.004796,2024-06-05,False


# Nettoyage des Données

In [37]:
import pandas as pd
import numpy as np
from datetime import datetime



# Vérifier les valeurs manquantes
print("Valeurs manquantes dans df_Aeronefs :\n", df_Aeronefs.isnull().sum())
print("Valeurs manquantes dans df_composants :\n", df_composants.isnull().sum())
print("Valeurs manquantes dans df_logs_vols :\n", df_logs_vols.isnull().sum())
print("Valeurs manquantes dans df_degradations :\n", df_degradations.isnull().sum())

Valeurs manquantes dans df_Aeronefs :
 ref_aero            0
type_model          0
debut_service       0
last_maint          0
en_maintenance      0
end_maint         193
dtype: int64
Valeurs manquantes dans df_composants :
 ref_compo            0
categorie            0
aero                 0
desc                 0
lifespan             0
taux_usure_actuel    0
cout                 0
dtype: int64
Valeurs manquantes dans df_logs_vols :
 ref_vol        0
aero_linked    0
jour_vol       0
time_en_air    0
sensor_data    0
etat_voyant    0
dtype: int64
Valeurs manquantes dans df_degradations :
 ref_deg             0
linked_aero         0
compo_concerned     0
usure_nouvelle      0
measure_day         0
need_replacement    0
dtype: int64


In [44]:
# Conversion des colonnes de date
# Remplissage des valeurs manquantes avec des dates par défaut ou des indicateurs de valeurs manquantes
df_Aeronefs['end_maint'] = pd.to_datetime(df_Aeronefs['end_maint'], errors='coerce', format='mixed')
df_Aeronefs['last_maint'] = pd.to_datetime(df_Aeronefs['last_maint'], errors='coerce', format='mixed')
df_logs_vols['jour_vol'] = pd.to_datetime(df_logs_vols['jour_vol'], errors='coerce', format='mixed')
df_degradations['measure_day'] = pd.to_datetime(df_degradations['measure_day'], errors='coerce', format='mixed')

In [45]:
df_Aeronefs

,ref_aero,type_model,debut_service,last_maint,en_maintenance,end_maint
0,E175_4124,E175,2003-10-01,2023-11-01,False,1900-01-01 00:00:00
1,E175_6334,E175,2010-02-18,2023-11-01,True,2023-11-08 14:13:00
2,B777_1214,B777,2003-10-21,2023-11-06,True,2023-11-13 14:13:00
3,CRJ900_0813,CRJ900,2002-08-01,2023-11-03,True,2023-11-10 14:13:00
4,A340_0268,A340,2003-01-12,2023-11-04,False,1900-01-01 00:00:00
...,...,...,...,...,...,...
227,B737_4516,B737,2001-10-07,2023-10-08,False,1900-01-01 00:00:00
228,B747_2319,B747,2002-09-27,2023-10-25,False,1900-01-01 00:00:00
229,A330_6228,A330,2012-05-23,2023-10-22,False,1900-01-01 00:00:00
230,A350_1769,A350,2022-03-20,2023-11-01,True,2023-11-08 14:13:00


In [53]:
import json

# Fonction pour extraire les données des capteurs
def extract_sensor_data(data):
    try:
        sensor_data = json.loads(data)
        temp = sensor_data.get('temp')
        pressure = sensor_data.get('pressure')
        vibration = sensor_data.get('vibration')
        return temp, pressure, vibration
    except (json.JSONDecodeError, KeyError):
        return None, None, None

# Extraire les données des capteurs
df_logs_vols['temp'], df_logs_vols['pressure'], df_logs_vols['vibration'] = zip(*df_logs_vols['sensor_data'].apply(extract_sensor_data))

In [55]:
# Afficher les premières lignes du DataFrame avec les données des capteurs
print(df_logs_vols[['temp', 'pressure', 'vibration']])

        temp    pressure vibration
0     17.8°C   904.2 hPa      None
1     -1.3°C  1029.6 hPa      None
2     16.4°C  1071.7 hPa      None
3    -24.3°C   905.9 hPa      None
4      5.3°C  1053.8 hPa      None
..       ...         ...       ...
134   -7.2°C   927.0 hPa      None
135    2.8°C   867.2 hPa      None
136  -10.2°C   998.9 hPa      None
137    5.0°C   932.7 hPa      None
138  -13.5°C   954.6 hPa      None

[139 rows x 3 columns]


In [56]:
df_logs_vols

,ref_vol,aero_linked,jour_vol,time_en_air,sensor_data,etat_voyant,temp,pressure,vibration
0,V06683852,B737_0940,2024-06-05,0.6,"{""temp"": ""17.8°C"", ""pressure"": ""904.2 hPa"", ""v...",0,17.8°C,904.2 hPa,None
1,V03551025,E175_6879,2024-06-05,5.2,"{""temp"": ""-1.3°C"", ""pressure"": ""1029.6 hPa"", ""...",0,-1.3°C,1029.6 hPa,None
2,V04512442,A320_2562,2024-06-05,7.9,"{""temp"": ""16.4°C"", ""pressure"": ""1071.7 hPa"", ""...",0,16.4°C,1071.7 hPa,None
3,V06831692,A320_1884,2024-06-05,7.4,"{""temp"": ""-24.3°C"", ""pressure"": ""905.9 hPa"", ""...",1,-24.3°C,905.9 hPa,None
4,V05264659,A321_3033,2024-06-05,3.6,"{""temp"": ""5.3°C"", ""pressure"": ""1053.8 hPa"", ""v...",0,5.3°C,1053.8 hPa,None
...,...,...,...,...,...,...,...,...,...
134,V01078793,CRJ700_2863,2024-06-05,7.4,"{""temp"": ""-7.2°C"", ""pressure"": ""927.0 hPa"", ""v...",0,-7.2°C,927.0 hPa,None
135,V00538128,A321_0181,2024-06-05,2.8,"{""temp"": ""2.8°C"", ""pressure"": ""867.2 hPa"", ""vi...",0,2.8°C,867.2 hPa,None
136,V00228114,E170_2583,2024-06-05,5.5,"{""temp"": ""-10.2°C"", ""pressure"": ""998.9 hPa"", ""...",0,-10.2°C,998.9 hPa,None
137,V09501055,A350_4039,2024-06-05,5.6,"{""temp"": ""5.0°C"", ""pressure"": ""932.7 hPa"", ""vi...",0,5.0°C,932.7 hPa,None


In [58]:
import json

# Fonction pour remplacer les guillemets simples par des guillemets doubles dans une chaîne JSON
def fix_json(json_str):
    return json_str.replace("'", '"')

# Extraire les données des capteurs en utilisant la fonction fix_json
df_logs_vols['sensor_data'] = df_logs_vols['sensor_data'].apply(fix_json)

# Maintenant, vous pouvez extraire les informations de température et de pression
df_logs_vols['temp'] = df_logs_vols['sensor_data'].apply(lambda x: json.loads(x)['temp'] if isinstance(x, str) else None)
df_logs_vols['pressure'] = df_logs_vols['sensor_data'].apply(lambda x: json.loads(x)['pressure'] if isinstance(x, str) else None)

In [59]:
df_logs_vols

,ref_vol,aero_linked,jour_vol,time_en_air,sensor_data,etat_voyant,temp,pressure,vibration
0,V06683852,B737_0940,2024-06-05,0.6,"{""temp"": ""17.8°C"", ""pressure"": ""904.2 hPa"", ""v...",0,17.8°C,904.2 hPa,None
1,V03551025,E175_6879,2024-06-05,5.2,"{""temp"": ""-1.3°C"", ""pressure"": ""1029.6 hPa"", ""...",0,-1.3°C,1029.6 hPa,None
2,V04512442,A320_2562,2024-06-05,7.9,"{""temp"": ""16.4°C"", ""pressure"": ""1071.7 hPa"", ""...",0,16.4°C,1071.7 hPa,None
3,V06831692,A320_1884,2024-06-05,7.4,"{""temp"": ""-24.3°C"", ""pressure"": ""905.9 hPa"", ""...",1,-24.3°C,905.9 hPa,None
4,V05264659,A321_3033,2024-06-05,3.6,"{""temp"": ""5.3°C"", ""pressure"": ""1053.8 hPa"", ""v...",0,5.3°C,1053.8 hPa,None
...,...,...,...,...,...,...,...,...,...
134,V01078793,CRJ700_2863,2024-06-05,7.4,"{""temp"": ""-7.2°C"", ""pressure"": ""927.0 hPa"", ""v...",0,-7.2°C,927.0 hPa,None
135,V00538128,A321_0181,2024-06-05,2.8,"{""temp"": ""2.8°C"", ""pressure"": ""867.2 hPa"", ""vi...",0,2.8°C,867.2 hPa,None
136,V00228114,E170_2583,2024-06-05,5.5,"{""temp"": ""-10.2°C"", ""pressure"": ""998.9 hPa"", ""...",0,-10.2°C,998.9 hPa,None
137,V09501055,A350_4039,2024-06-05,5.6,"{""temp"": ""5.0°C"", ""pressure"": ""932.7 hPa"", ""vi...",0,5.0°C,932.7 hPa,None


In [61]:
# Convertir les valeurs de température en nombres
df_logs_vols['temp'] = pd.to_numeric(df_logs_vols['temp'], errors='coerce')

# Supprimer les valeurs aberrantes pour les températures
df_logs_vols.loc[(df_logs_vols['temp'] < -50) | (df_logs_vols['temp'] > 50), 'temp'] = np.nan

In [62]:
df_logs_vols

,ref_vol,aero_linked,jour_vol,time_en_air,sensor_data,etat_voyant,temp,pressure,vibration
0,V06683852,B737_0940,2024-06-05,0.6,"{""temp"": ""17.8°C"", ""pressure"": ""904.2 hPa"", ""v...",0,NaN,904.2 hPa,None
1,V03551025,E175_6879,2024-06-05,5.2,"{""temp"": ""-1.3°C"", ""pressure"": ""1029.6 hPa"", ""...",0,NaN,1029.6 hPa,None
2,V04512442,A320_2562,2024-06-05,7.9,"{""temp"": ""16.4°C"", ""pressure"": ""1071.7 hPa"", ""...",0,NaN,1071.7 hPa,None
3,V06831692,A320_1884,2024-06-05,7.4,"{""temp"": ""-24.3°C"", ""pressure"": ""905.9 hPa"", ""...",1,NaN,905.9 hPa,None
4,V05264659,A321_3033,2024-06-05,3.6,"{""temp"": ""5.3°C"", ""pressure"": ""1053.8 hPa"", ""v...",0,NaN,1053.8 hPa,None
...,...,...,...,...,...,...,...,...,...
134,V01078793,CRJ700_2863,2024-06-05,7.4,"{""temp"": ""-7.2°C"", ""pressure"": ""927.0 hPa"", ""v...",0,NaN,927.0 hPa,None
135,V00538128,A321_0181,2024-06-05,2.8,"{""temp"": ""2.8°C"", ""pressure"": ""867.2 hPa"", ""vi...",0,NaN,867.2 hPa,None
136,V00228114,E170_2583,2024-06-05,5.5,"{""temp"": ""-10.2°C"", ""pressure"": ""998.9 hPa"", ""...",0,NaN,998.9 hPa,None
137,V09501055,A350_4039,2024-06-05,5.6,"{""temp"": ""5.0°C"", ""pressure"": ""932.7 hPa"", ""vi...",0,NaN,932.7 hPa,None


In [60]:
# Supposons que les valeurs aberrantes pour les températures sont au-delà des plages habituelles (-50°C à 50°C)
df_logs_vols.loc[(df_logs_vols['temp'] < -50) | (df_logs_vols['temp'] > 50), 'temp'] = np.nan

# De même pour les pressions et vibrations avec des seuils arbitraires que nous définirons :
df_logs_vols.loc[(df_logs_vols['pressure'] < 800) | (df_logs_vols['pressure'] > 1100), 'pressure'] = np.nan
df_logs_vols.loc[(df_logs_vols['vibration'] < 0) | (df_logs_vols['vibration'] > 100), 'vibration'] = np.nan

TypeError: '<' not supported between instances of 'str' and 'int'

In [ ]:
import numpy as np
#Gestion des valeurs aberrantes :
# Supposons que les valeurs aberrantes pour les températures sont au-delà des plages habituelles (-50°C à 50°C)
df_logs_vols.loc[(df_logs_vols['temp'] < -50) | (df_logs_vols['temp'] > 50), 'temp'] = np.nan

# De même pour les pressions et vibrations avec des seuils arbitraires que nous définirons :
df_logs_vols.loc[(df_logs_vols['pressure'] < 800) | (df_logs_vols['pressure'] > 1100), 'pressure'] = np.nan
df_logs_vols.loc[(df_logs_vols['vibration'] < 0) | (df_logs_vols['vibration'] > 100), 'vibration'] = np.nan

In [63]:
#Nettoyage des données de composants :
# S'assurer que le taux d'usure ne dépasse pas 100 %
df_composants['taux_usure_actuel'] = df_composants['taux_usure_actuel'].apply(lambda x: min(x, 100))

In [64]:
df_composants

,ref_compo,categorie,aero,desc,lifespan,taux_usure_actuel,cout
0,REAE175-E175_4124-0,Composants Critiques,E175_4124,Réacteur gauche,11950,69.234150,19845
1,REAE175-E175_4124-1,Composants Critiques,E175_4124,Réacteur droit,14014,65.181958,17416
2,SYSE175-E175_4124-2,Composants Critiques,E175_4124,Système de navigation,12519,17.981443,16184
3,ORDE175-E175_4124-3,Composants Critiques,E175_4124,Ordinateur de vol,11612,26.147164,18574
4,AUTE175-E175_4124-4,Composants Critiques,E175_4124,Autopilote,10655,11.473035,18769
...,...,...,...,...,...,...,...
10435,SYSB767-B767_6363-40,Composants Secondaires,B767_6363,Systèmes d'oxygène d'urgence,12602,26.476842,4344
10436,PORB767-B767_6363-41,Composants Secondaires,B767_6363,Portes passagers,11227,19.875467,2970
10437,HUBB767-B767_6363-42,Composants Secondaires,B767_6363,Hublots,14930,17.766530,2567
10438,ECLB767-B767_6363-43,Composants Secondaires,B767_6363,Éclairage de la cabine,11837,28.869277,4034


In [ ]:
#Combinaison des données :
#Création de DataFrames combinés pour l'analyse des composants en relation avec les aéronefs et les logs de vol.
# Combinaison des données des composants avec les données des aéronefs
df_composants_aeronefs = pd.merge(df_composants, df_aeronefs, left_on='aero', right_on='ref_aero', how='left')

# Combinaison des logs de vols avec les données des aéronefs
df_logs_vols_aeronefs = pd.merge(df_logs_vols, df_aeronefs, left_on='aero_linked', right_on='ref_aero', how='left')

# Affichage des premières lignes des DataFrames combinés pour vérifier la fusion
print(df_composants_aeronefs.head())
print(df_logs_vols_aeronefs.head())